In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/workspaces/SebastianBot")

In [2]:
from cloud.dependencies.clients import resolve_gmail_client


gmail = resolve_gmail_client()

In [14]:
from datetime import datetime
from sebastian.shared.gmail.query_builder import GmailQueryBuilder


mails = gmail.fetch_mails(GmailQueryBuilder().from_email("no-reply@doctolib.de").after_date(datetime(2025, 12, 28)).build())
mail = mails[0]
mail

FullMailResponse(id='19b695ce02320366', threadId='19b695ce02320366', labelIds=['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX'], snippet='Doctolib Ihr Termin musste abgesagt werden Dienstag 13 Januar 07:30 E. Louka Hallo Oliver, leider musste die Gesundheitseinrichtung Ihren Termin absagen. NEUEN TERMIN FINDEN Kontakt Über uns', raw_payload={'partId': '', 'mimeType': 'multipart/alternative', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'oneironaut.oml@gmail.com'}, {'name': 'Received', 'value': 'by 2002:ac4:d1ca:0:b0:763:e3e5:a35f with SMTP id n10csp5626752pia;        Mon, 29 Dec 2025 01:07:33 -0800 (PST)'}, {'name': 'X-Google-Smtp-Source', 'value': 'AGHT+IG8k881s7O/9ypP2M0eMLbzaNXo8+oVz8v5QSnuiSiDlFrzc1WRKJOHIzPw/mAGSr0XTo9t'}, {'name': 'X-Received', 'value': 'by 2002:a05:6000:22c1:b0:431:771:a50c with SMTP id ffacd0b85a97d-432447952a3mr39064114f8f.1.1766999253505;        Mon, 29 Dec 2025 01:07:33 -0800 (PST)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1766999253; 

In [15]:
html_string =mail.payload
html_string

'<!DOCTYPE html><html lang="de"><head data-country="de" data-env="production"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"/><title>Doctolib</title><style type="text/css">a {\r\n  text-decoration: none;\r\n  color: #107ACA;\r\n}\r\n.footer-text a {\r\n  text-decoration: underline;\r\n  color: #00264C;\r\n}\r\nbody {\r\n  font-family: Arial, sans-serif;\r\n  font-size: 14px;\r\n  background-color: #E4EDF6;\r\n}</style></head><body bgcolor="#E4EDF6" leftmargin="0" marginheight="0" marginwidth="0" style="font-family: Arial, sans-serif" topmargin="0"><table align="center" bgcolor="#E4EDF6" border="0" cellpadding="0" cellspacing="0" role="presentation" width="100%"><tbody><tr height="14"><td></td><td></td><td></td></tr><tr><td width="14"></td><td align="center"><table width="100%" border="0" cellpadding="0" cellspacing="0" style="max-width: 650px;" role="presentation"><tbody><tr><td><table bgcolor="#FFFFFF" border="0" cellpadding="0" cellspacing="0" role="presentation" 

In [16]:
from bs4 import BeautifulSoup

# Parse the HTML string and extract visible text
soup = BeautifulSoup(html_string, "html.parser")
visible_text = soup.get_text()
visible_text

'DoctolibIhr Termin musste abgesagt werdenDienstag 13 Januar07:30E. LoukaHallo Oliver,leider musste die Gesundheitseinrichtung Ihren Termin absagen.NEUEN TERMIN FINDENKontaktÜber unsHilfebereichDiese E-Mail wird Ihnen durch Doctolib im Auftrag und im Namen Ihrer Gesundheitsfachkraft zugesandt, da Ihre Gesundheitsfachkraft die Dienste von Doctolib nutzt, um ihre Termine zu verwalten. Doctolib handelt als Auftragsverarbeiter und Ihre Gesundheitsfachkraft als verantwortliche Stelle.\r\nUm Ihre Rechte in Bezug auf Ihre persönlichen Daten auszuüben, können Sie sich direkt an Ihre sendende Gesundheitsfachkraft wenden. \r\nUm mehr darüber zu erfahren, wie Doctolib Ihre personenbezogenen Daten verarbeitet, lesen Sie bitte unsere   Datenschutzhinweise.\r\nDiese E-Mail wird durch Doctolib GmbH, Mehringdamm 51, 10961 Berlin, Amtsgericht Charlottenburg HRB 175963B, gesendet.\r\nBitte beachten Sie, dass dies nicht der Ort ist, an dem Ihr Termin stattfindet.\n\n'

In [18]:
from cloud.dependencies.clients import resolve_gemini_client


gemini = resolve_gemini_client()

In [25]:
prompt = f"""You are my personal assistant, managing incoming emails.
Your goal is to decide if an email requires my attention or can be ignored.
You have the following options:
1. Create a task in my calendar. This should be used whenever the mail content means that I need to do something. I.e. attend an appointment, make a call, send an email, or reschedule an appointment.
2. Send a telegram message. This should be used if the mail content is important, but creating a task does not make sense.

You may chose to any number and any combination of the above options, or none at all.
Reply in german.

---Mail content (text only)---
{visible_text}
---End of mail content---"""

In [26]:
from cloud.functions.infrastructure.AllHandler.models import AllHandlerEvent


response = gemini.get_response(prompt, AllHandlerEvent)

In [27]:
response

Result(item=create_task_events=[CreateTaskEvent(title='Neuen Doctolib-Termin finden', notes='Der Termin am Dienstag, 13. Januar um 07:30 Uhr wurde abgesagt.', due=None, task_list_id=<TaskListIds.Default: 'MDY0Mzc2NjgyMDc4MTc0Nzg3Mjk6MDow'>)] send_telegram_message_events=[], errors=[])